In [152]:
import pandas as pd
import numpy as np

df = pd.read_excel("ALL_HR_ALL.xlsx") # relative path to input file here

# df
# rcid, ccode, vote, countyname, year
newdf = df.drop(["member", "Country", "session", "abstain", "yes", 
                 "no", "date", "unres", "amend", "para", "short", 
                 "descr", "hr", "resid", "importantvote", "co"], axis='columns')
newdf["Countryname"] = newdf["Countryname"].str.lower()
newdf

,rcid,ccode,vote,Countryname,year
0,4593,2,3,united states of america,2006
1,4593,20,1,canada,2006
2,4593,31,1,bahamas,2006
3,4593,40,1,cuba,2006
4,4593,41,1,haiti,2006
...,...,...,...,...,...
66773,9312,970,3,nauru,2021
66774,9312,983,3,marshall islands,2021
66775,9312,986,1,palau,2021
66776,9312,987,3,micronesia (federated states of),2021


A value of 1 to countries that vote the same way as China; a value of 0.5 if one of the countries decided to abstain while the other voted either a yes or no; and finally a 0 for countries that voted the opposite to China’s decision. A yearly average was then calculated by dividing a country’s total voting convergence score by the total number of country-specific human rights votes in that year, converting the final value into percentage terms by applying a natural logarithmic transformation.

In [153]:
# list of rcids
rcids = []
for i in newdf["rcid"]:
    if i not in rcids:
        rcids.append(i)

In [154]:
newdf["cvote"] = 0

cvotes = []

resgroup = newdf.groupby(newdf.rcid)
# df1 = resgroup.get_group(4723)

for i in range(len(rcids)):
    df1 = resgroup.get_group(rcids[i])
    for q in range(len(df1)):
        if df1.iloc[q]["ccode"] == 710:
            cvotes.append(df1.iloc[q]["vote"])

print(len(rcids))
print(len(cvotes))
dictionary = {}
for i in range(len(rcids)):
    dictionary[rcids[i]] = cvotes[i]
    
print(dictionary)

346
346
{4593: 1, 4673: 1, 4674: 1, 4705: 1, 4717: 1, 4718: 1, 4719: 1, 4720: 1, 4723: 3, 4724: 3, 4725: 3, 4727: 1, 4728: 1, 4729: 1, 4731: 1, 4732: 1, 4734: 1, 4735: 1, 4736: 1, 4737: 1, 4738: 1, 4739: 1, 4740: 1, 4741: 1, 4747: 2, 4749: 3, 5798: 1, 5799: 3, 4753: 1, 4780: 1, 4781: 1, 4789: 1, 4790: 1, 4795: 1, 4796: 1, 4798: 1, 4799: 1, 4800: 1, 4801: 3, 4802: 1, 4803: 1, 4804: 1, 4805: 1, 4806: 1, 4807: 3, 4808: 1, 4809: 1, 4810: 1, 4811: 1, 4812: 1, 4813: 3, 4814: 3, 4820: 3, 4825: 1, 4827: 1, 4839: 1, 4840: 1, 4874: 1, 4875: 1, 4878: 1, 4880: 1, 4881: 1, 4882: 3, 4883: 1, 4884: 1, 4885: 1, 4886: 2, 4887: 1, 4888: 1, 4889: 3, 4890: 3, 4891: 1, 4892: 1, 4893: 1, 4894: 1, 4895: 1, 4896: 1, 4897: 1, 4902: 3, 4903: 1, 4904: 1, 4980: 1, 4981: 1, 4982: 1, 5011: 1, 5012: 1, 5013: 1, 5020: 1, 5021: 1, 5023: 1, 5024: 1, 5025: 1, 5026: 1, 5027: 1, 5028: 1, 5029: 3, 5030: 3, 5031: 1, 5032: 1, 5033: 1, 5034: 1, 5035: 1, 5036: 1, 5043: 3, 5044: 1, 5046: 1, 5052: 1, 5053: 1, 5082: 1, 5083: 1, 5

In [155]:
for i in range(len(newdf)):
    newdf.loc[i, "cvote"] = dictionary[newdf.loc[i]["rcid"]]
    
print(newdf)


       rcid  ccode  vote                       Countryname  year  cvote
0      4593      2     3          united states of america  2006      1
1      4593     20     1                            canada  2006      1
2      4593     31     1                           bahamas  2006      1
3      4593     40     1                              cuba  2006      1
4      4593     41     1                             haiti  2006      1
...     ...    ...   ...                               ...   ...    ...
66773  9312    970     3                             nauru  2021      1
66774  9312    983     3                  marshall islands  2021      1
66775  9312    986     1                             palau  2021      1
66776  9312    987     3  micronesia (federated states of)  2021      1
66777  9312    990     1                             samoa  2021      1

[66778 rows x 6 columns]


In [156]:
newdf["alignment"] = "NA"

for i in range(len(newdf)):
    if (newdf.loc[i, "vote"] == 8) or (newdf.loc[i, "vote"] == 9):
        newdf.loc[i, "alignment"] = -100000
    elif newdf.loc[i, "vote"] == newdf.loc[i, "cvote"]:
        newdf.loc[i, "alignment"] = 1
    elif (newdf.loc[i, "vote"] == 2) or (newdf.loc[i, "cvote"] == 2):
        newdf.loc[i, "alignment"] = 0.5
    else:
        newdf.loc[i, "alignment"] = 0

In [157]:
balls = pd.DataFrame( columns =['Country', 'Year', 'Alignment', 'num_resos'])

Countrynames = []
for i in newdf["Countryname"]:
    if i not in Countrynames:
        Countrynames.append(i)
        
print(Countrynames)

['united states of america', 'canada', 'bahamas', 'cuba', 'haiti', 'dominican republic', 'jamaica', 'trinidad and tobago', 'barbados', 'dominica', 'grenada', 'saint lucia', 'saint vincent and the grenadines', 'antigua and barbuda', 'saint kitts and nevis', 'mexico', 'belize', 'guatemala', 'honduras', 'el salvador', 'nicaragua', 'costa rica', 'panama', 'colombia', 'venezuela, bolivarian republic of', 'guyana', 'suriname', 'ecuador', 'peru', 'brazil', 'bolivia (plurinational state of)', 'paraguay', 'chile', 'argentina', 'uruguay', 'united kingdom of great britain and northern ireland', 'ireland', 'netherlands', 'belgium', 'luxembourg', 'france', 'monaco', 'liechtenstein', 'switzerland', 'spain', 'andorra', 'portugal', 'germany', 'german democratic republic', 'poland', 'austria', 'hungary', 'czechoslovakia', 'czech republic', 'slovakia', 'italy', 'san marino', 'malta', 'albania', 'the former yugoslav republic of macedonia', 'croatia', 'yugoslavia', 'bosnia and herzegovina', 'slovenia', 'g

In [158]:

        

resgroup = newdf.groupby(newdf.Countryname)

for i in range(len(Countrynames)):
    df1 = resgroup.get_group(Countrynames[i])
    years = []
    for q in range(len(df1)):
        if df1.iloc[q]["year"] not in years:
            years.append(df1.iloc[q]["year"])
    subgroup = df1.groupby(df1.year)
    for r in range(len(years)):
        df2 = subgroup.get_group(years[r])
        sum = 0
        numres = 0
        for s in range(len(df2)):
            if (df2.iloc[s]["vote"]!= 8) and (df2.iloc[s]["vote"]!= 9):
                sum += df2.iloc[s]["alignment"]
                numres += 1
        if numres == 0:
            average = 88
        else:
            average = sum / numres
        
        app = pd.DataFrame({"Country": Countrynames[i],
                            "Year": years[r],
                            "Alignment": average,
                            "num_resos" : numres},index=[0])
        balls = pd.concat([balls, app], ignore_index=True)

In [ ]:
balls.to_excel('final.xlsx') # intended name  here